In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet import Prophet
import logging
import warnings
from datetime import datetime
import fbprophet
import pystan

warnings.filterwarnings('ignore')
logging.getLogger('fbprophet').setLevel(logging.WARNING)

Importing plotly failed. Interactive plots will not work.


In [2]:
#데이터 불러오기
df_join = pd.read_csv("sales_member_join_processed.csv")
df_member = pd.read_csv("jeju_member.csv")
df_sales = pd.read_csv("jeju_sales_processed.csv")

-------

# 데이터 만들기 (볼필요x)

In [3]:
df_merge = pd.merge(df_member, df_sales)

In [4]:
df_merge_grouped = df_merge.groupby("customer_id")

In [5]:
df_merge_grouped = df_merge_grouped.sum().reset_index()

In [6]:
df_merge_grouped.head()

,customer_id,age,member_month,purchase_hour,purchase_cnt,unit_cost,purchase_amt
0,C118113,405,873,116,13,74500,99500
1,C118291,9512,15908,2550,180,984200,1062850
2,C118294,1222,2522,361,38,125000,165000
3,C118295,928,1552,226,16,107600,107600
4,C118298,9792,13968,2040,174,457800,526090


In [7]:
df_merge_grouped = df_merge_grouped[["customer_id", "purchase_amt", "purchase_cnt"]]

In [8]:
df_merge_grouped

,customer_id,purchase_amt,purchase_cnt
0,C118113,99500,13
1,C118291,1062850,180
2,C118294,165000,38
3,C118295,107600,16
4,C118298,526090,174
...,...,...,...
2750,C704198,49850,30
2751,C704239,18900,3
2752,C704271,43800,11
2753,C704612,10800,4


In [9]:
df_semi = pd.merge(df_merge_grouped, df_member, left_on="customer_id", right_on="customer_id")

In [16]:
df_semi.head()

,customer_id,purchase_amt,purchase_cnt,birth_date,gender,age,agegroup,customer_type,customer_grade,gaip_date,...,gaip_path,flag_member,flag_rcv_sms,flag_rcv_email,flag_rcv_post,zipcode,sigun,dong,ri,address
0,C118113,99500,13,1975-08-18,여,45,40대,일반,3Gold,2011-12-06,...,기타,Y,Y,Y,Y,690-122,제주시,아라2동,NaN,제주 제주시 아라2동
1,C118291,1062850,180,1962-07-19,여,58,50대,일반,1Royal,2011-12-11,...,기타,Y,Y,N,Y,690-760,제주시,노형동,부영아파트,제주 제주시 노형동 부영아파트
2,C118294,165000,38,1973-04-19,여,47,40대,일반,2Platinum,2011-12-11,...,기타,Y,Y,Y,Y,697-835,서귀포시,동홍동,119~434,제주 서귀포시 동홍동 119~434
3,C118295,107600,16,1962-07-23,여,58,50대,일반,3Gold,2011-12-11,...,매장,Y,Y,Y,Y,690-802,제주시,노형동,1~1019,제주 제주시 노형동 1~1019
4,C118298,526090,174,1952-01-12,여,68,60대,일반,2Platinum,2011-12-11,...,전화,Y,N,Y,Y,699-930,서귀포시,대정읍,NaN,제주 서귀포시 대정읍


-----

# 이상한 데이터들 (왜 이게 Royal?Platinum?Gold?)

In [10]:
df_final = df_semi[["customer_id", "purchase_amt", "purchase_cnt", "customer_grade"]]

In [17]:
#df_royal
df_royal = df_final[df_final["customer_grade"]=="1Royal"]
df_royal.sort_values(by=['purchase_amt'], axis=0).head(10)

,customer_id,purchase_amt,purchase_cnt,customer_grade
799,C450393,8600,2,1Royal
808,C450436,15450,5,1Royal
2005,C597307,21000,7,1Royal
407,C354460,21200,3,1Royal
353,C308961,35800,3,1Royal
426,C364526,77100,23,1Royal
162,C164701,87500,21,1Royal
803,C450409,127600,22,1Royal
6,C118300,184800,33,1Royal
191,C222499,341400,40,1Royal


In [18]:
#df_platinum
df_platinum = df_final[df_final["customer_grade"]=="2Platinum"]
df_platinum.sort_values(by=['purchase_amt'], axis=0).head(10)

,customer_id,purchase_amt,purchase_cnt,customer_grade
2317,C639452,1300,1,2Platinum
1909,C585655,1900,1,2Platinum
2135,C613163,5100,3,2Platinum
780,C450280,5350,3,2Platinum
2651,C689101,5700,3,2Platinum
1965,C593153,6000,1,2Platinum
820,C450485,6400,3,2Platinum
2712,C700399,6700,2,2Platinum
2686,C695326,6900,3,2Platinum
264,C240140,7600,2,2Platinum


In [19]:
#df_gold
df_gold = df_final[df_final["customer_grade"]=="3Gold"]
df_gold.sort_values(by=['purchase_amt'], axis=0).head(10)

,customer_id,purchase_amt,purchase_cnt,customer_grade
2139,C613594,1000,1,3Gold
1672,C553056,1300,1,3Gold
2304,C638199,1300,1,3Gold
585,C418980,1300,1,3Gold
1274,C489663,1300,1,3Gold
637,C431361,1300,1,3Gold
519,C395678,1350,1,3Gold
1099,C475740,1500,1,3Gold
1945,C590376,1600,1,3Gold
2667,C692530,1600,1,3Gold


In [20]:
#df_silver
df_silver = df_final[df_final["customer_grade"]=="4Silver"]
df_silver.sort_values(by=['purchase_amt'], axis=0).head(10)

,customer_id,purchase_amt,purchase_cnt,customer_grade
949,C463879,1300,1,4Silver
628,C428847,1400,1,4Silver
214,C225278,1550,1,4Silver
1490,C523159,1700,1,4Silver
1914,C586273,1700,1,4Silver
669,C438491,1800,1,4Silver
1494,C523809,1800,1,4Silver
2112,C610370,2000,2,4Silver
2457,C657979,2100,1,4Silver
1771,C565224,2200,1,4Silver
